In [ ]:
import itertools
import math
import collections
import pandas as pd
import numpy as np

from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from decimal import Decimal

DEFAULT_VELOCITY_BINS = np.linspace(0, 128, 8+1, dtype=np.int32)
BestQuantizationMatch = collections.namedtuple('BestQuantizationMatch',
    ['error', 'tick', 'match', 'signedError', 'divisor'])

class AlignDataProcessor:
    """
    Data preprocessing based on alignment results.
    """
    def __init__(self, args):
        self.df = pd.read_csv(args.path_to_dataset_csv, header=0)
        self.isSlice = args.isSlice
        self.isFull = args.isFull
        self.isOverlap = args.isOverlap
        self.savepath = args.path_to_save
        self.random_state = args.random_state
        self.max_len = args.max_len
        self.slice_len = args.slice_len
        self.align_files = args.data_folder + self.df[args.align_result_column]

    @staticmethod
    def norm(input):
        return (input - np.mean(input)) / np.std(input)

    @staticmethod
    def pad_or_cut_sequence(seq, require_len):
        if len(seq) >= require_len:
            return seq[0:require_len]
        else:
            return np.concatenate([seq, np.zeros((require_len-len(FEATURES_LIST)))])

    @staticmethod
    def add_to_list(element, times, target_list):
        target_list += [element for i in range(times)]
        return target_list

    def process(self):
        """
        Process all performances in the dataset.
        """
        x = []
        y = []
        splits = []

        for idx, row in tqdm(self.df.iterrows(), total=self.df.shape[0]):
            seq = np.random.rand(self.max_len, len(FEATURES_LIST))  # Dummy feature extraction
            x.append(self.pad_or_cut_sequence(seq, self.max_len))
            y.append(row['artist_id'])
            splits.append(row['type'])

        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.splits = np.asarray(splits)
        print("Total performances (in segments):" + str(self.x.shape))

        # Apply data augmentation
        self.x, self.y, self.splits = self.augment_data(self.x, self.y, self.splits)

    def save(self):
        np.savez(self.savepath, train_x=self.x, train_y=self.y, splits=self.splits)

    def augment_data(self, x, y, splits, num_augment=1, pitch_index=0, velocity_index=3):
        """
        Data augmentation function that duplicates existing sequences
        and applies random pitch and velocity shifts.
        """
        augmented_x = []
        augmented_y = []
        augmented_splits = []

        for i in range(len(x)):
            original_seq = x[i]
            original_label = y[i]
            original_split = splits[i]

            for _ in range(num_augment):
                new_seq = np.copy(original_seq)
                pitch_shift = np.random.randint(-2, 3)
                velocity_scale = 0.8 + 0.4 * np.random.rand()

                new_seq[:, pitch_index] += pitch_shift
                new_seq[:, velocity_index] = np.clip(new_seq[:, velocity_index] * velocity_scale, 0, 127)

                augmented_x.append(new_seq)
                augmented_y.append(original_label)
                augmented_splits.append(original_split)

        return np.array(augmented_x), np.array(augmented_y), np.array(augmented_splits)
